In [1]:
import pandas as pd
import requests
from tqdm.auto import tqdm
tqdm.pandas()

In [2]:
def next_monday(date):
    return pd.date_range(start=date, end=date + pd.offsets.Day(6), freq='W-MON')[0]

# Setting

In [3]:
test_date = pd.to_datetime('2021-03-22')

In [16]:
test_date = pd.to_datetime('2021-04-12')

In [87]:
models_to_exclude = ['COVIDhub-ensemble', 'COVIDhub-trained_ensemble', 
                     'CU-nochange', 'CU-scenario_high', 'CU-scenario_low', 'CU-scenario_mid']

locations_to_exclude = ["11", "60", "66", "69", "72", "74", "78"]

# DC,11,District of Columbia
# AS,60,American Samoa
# GU,66,Guam
# MP,69,Northern Mariana Islands
# PR,72,Puerto Rico
# UM,74,U.S. Minor Outlying Islands
# VI,78,Virgin Islands

In [18]:
dtype={'target': str, 'location': str, 'type': str, 'quantile': float, 'value': float}
parse_dates=['forecast_date', 'target_end_date']

# Load Files

In [19]:
url = "https://api.github.com/repos/reichlab/covid19-forecast-hub/git/trees/master?recursive=1"
r = requests.get(url)
res = r.json()

In [20]:
files = [file["path"] for file in res["tree"] if (file["path"].startswith('data-processed/') and file["path"].endswith('.csv'))]

In [21]:
files[:5]

['data-processed/AIpert-pwllnod/2020-12-21-AIpert-pwllnod.csv',
 'data-processed/AIpert-pwllnod/2020-12-28-AIpert-pwllnod.csv',
 'data-processed/AIpert-pwllnod/2021-01-04-AIpert-pwllnod.csv',
 'data-processed/AIpert-pwllnod/2021-01-11-AIpert-pwllnod.csv',
 'data-processed/AIpert-pwllnod/2021-01-18-AIpert-pwllnod.csv']

In [22]:
df_files = pd.DataFrame({'filename':files})

df_files['model'] = df_files.filename.apply(lambda f: f.split('/')[1])

df_files['forecast_date'] = df_files.filename.apply(lambda f: f.split('/')[2][:10])
df_files.forecast_date = pd.to_datetime(df_files.forecast_date)

df_files['timezero'] = df_files.forecast_date.apply(next_monday)

df_files = df_files[~df_files.model.isin(models_to_exclude)]

In [23]:
df_files

,filename,model,forecast_date,timezero
0,data-processed/AIpert-pwllnod/2020-12-21-AIper...,AIpert-pwllnod,2020-12-21,2020-12-21
1,data-processed/AIpert-pwllnod/2020-12-28-AIper...,AIpert-pwllnod,2020-12-28,2020-12-28
2,data-processed/AIpert-pwllnod/2021-01-04-AIper...,AIpert-pwllnod,2021-01-04,2021-01-04
3,data-processed/AIpert-pwllnod/2021-01-11-AIper...,AIpert-pwllnod,2021-01-11,2021-01-11
4,data-processed/AIpert-pwllnod/2021-01-18-AIper...,AIpert-pwllnod,2021-01-18,2021-01-18
...,...,...,...,...
3100,data-processed/epiforecasts-ensemble1/2021-03-...,epiforecasts-ensemble1,2021-03-15,2021-03-15
3101,data-processed/epiforecasts-ensemble1/2021-03-...,epiforecasts-ensemble1,2021-03-22,2021-03-22
3102,data-processed/epiforecasts-ensemble1/2021-03-...,epiforecasts-ensemble1,2021-03-29,2021-03-29
3103,data-processed/epiforecasts-ensemble1/2021-04-...,epiforecasts-ensemble1,2021-04-05,2021-04-05


### Test Data

In [70]:
VALID_TARGETS = [f"{_} wk ahead inc death" for _ in range(1, 5)] + \
                [f"{_} wk ahead cum death" for _ in range(1, 5)] # + \
                # [f"{_} wk ahead inc case" for _ in range(1, 5)]

In [30]:
df_test_files = df_files[df_files.timezero == test_date]

In [31]:
df_test_files.head()

,filename,model,forecast_date,timezero
16,data-processed/AIpert-pwllnod/2021-04-12-AIper...,AIpert-pwllnod,2021-04-12,2021-04-12
56,data-processed/BPagano-RtDriven/2021-04-11-BPa...,BPagano-RtDriven,2021-04-11,2021-04-12
99,data-processed/CEID-Walk/2021-04-12-CEID-Walk.csv,CEID-Walk,2021-04-12,2021-04-12
138,data-processed/CMU-TimeSeries/2021-04-12-CMU-T...,CMU-TimeSeries,2021-04-12,2021-04-12
192,data-processed/COVIDhub-baseline/2021-04-12-CO...,COVIDhub-baseline,2021-04-12,2021-04-12


In [32]:
dfs_test = []

for _, row in tqdm(df_test_files.iterrows(), total=df_test_files.shape[0]):
    #print(row['filename'])
    df_temp = pd.read_csv('https://github.com/reichlab/covid19-forecast-hub/raw/master/' + row['filename'],
                         dtype=dtype, parse_dates=parse_dates)
    df_temp = df_temp[df_temp.target.isin(VALID_TARGETS)]
    df_temp['model'] = row['model']
    dfs_test.append(df_temp)

  0%|          | 0/57 [00:00<?, ?it/s]

In [33]:
df_test = pd.concat(dfs_test)

In [34]:
df_test.shape

(391430, 8)

In [35]:
df_test

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-04-12,1 wk ahead cum death,2021-04-17,0.025,quantile,565499.673705,US,AIpert-pwllnod
1,2021-04-12,1 wk ahead cum death,2021-04-17,0.250,quantile,566525.854459,US,AIpert-pwllnod
2,2021-04-12,1 wk ahead cum death,2021-04-17,0.750,quantile,567960.212535,US,AIpert-pwllnod
3,2021-04-12,1 wk ahead cum death,2021-04-17,0.975,quantile,569667.688730,US,AIpert-pwllnod
4,2021-04-12,1 wk ahead cum death,2021-04-17,NaN,point,567064.294607,US,AIpert-pwllnod
...,...,...,...,...,...,...,...,...
10939,2021-04-12,4 wk ahead cum death,2021-05-08,0.850,quantile,599324.000000,US,epiforecasts-ensemble1
10940,2021-04-12,4 wk ahead cum death,2021-05-08,0.900,quantile,600859.000000,US,epiforecasts-ensemble1
10941,2021-04-12,4 wk ahead cum death,2021-05-08,0.950,quantile,603234.000000,US,epiforecasts-ensemble1
10942,2021-04-12,4 wk ahead cum death,2021-05-08,0.975,quantile,605933.000000,US,epiforecasts-ensemble1


In [36]:
# only consider US + 50 states
df_test = df_test[df_test.location.str.len() == 2]
df_test = df_test[~df_test.location.isin(locations_to_exclude)]

In [37]:
df_test.shape

(372558, 8)

In [38]:
# ensure that for all targets each model provides forecasts for all locations
df_test = df_test[df_test.groupby(['target', 'model'])['location'].transform('nunique') == 51]

In [100]:
df_test = df_test[df_test.type == 'quantile']

In [104]:
df_test['no_quantiles'] = df_test.groupby(['model', 'target', 'target_end_date', 'location'])['quantile'].transform('nunique')

df_test['no_quantiles'] = df_test.groupby(['target', 'model'])['no_quantiles'].transform('min')

In [107]:
df_test = df_test[(df_test.no_quantiles == 23) | 
        (df_test.target.str.contains('inc case') & (df_test.no_quantiles == 7))].drop(columns='no_quantiles').reset_index(drop=True)

In [108]:
df_test.shape

(206448, 8)

In [40]:
# dict of the models available for each target
available_models = dict(df_test.groupby(['target'])['model'].unique())

In [41]:
available_models

{'1 wk ahead cum death': array(['AIpert-pwllnod', 'BPagano-RtDriven', 'CEID-Walk',
        'COVIDhub-baseline', 'CU-nochange', 'CU-scenario_high',
        'CU-scenario_low', 'CU-scenario_mid', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'DDS-NBDS',
        'Geneva-DetGrowth', 'JHUAPL-Bucky', 'JHU_CSSE-DECOM',
        'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1', 'MIT_CritData-GBCF',
        'MOBS-GLEAM_COVID', 'Microsoft-DeepSTIA', 'OliverWyman-Navigator',
        'RobertWalraven-ESG', 'SigSci-TS', 'SteveMcConnell-CovidComplete',
        'TTU-squider', 'UA-EpiCovDA', 'UCSD_NEU-DeepGLEAM',
        'UMass-MechBayes', 'USC-SI_kJalpha', 'UT-Mobility',
        'epiforecasts-ensemble1'], dtype=object),
 '1 wk ahead inc death': array(['AIpert-pwllnod', 'BPagano-RtDriven', 'CEID-Walk',
        'COVIDhub-baseline', 'CU-nochange', 'CU-scenario_high',
        'CU-scenario_low', 'CU-scenario_mid', 'CU-select', 'Caltech-CS156',
        'Columbia_UNC-SurvCon', 'Covid19Sim-S

## Training Data

d - 4 weeks - (horizon - 1) --> end: d - horizon

In [43]:
lower_bound = test_date - pd.Timedelta(weeks=4) - pd.Timedelta(weeks=(4 - 1))

In [44]:
lower_bound

Timestamp('2021-02-22 00:00:00')

In [45]:
df_files = df_files[(df_files.timezero >= lower_bound) & (df_files.timezero < test_date)].copy()

In [46]:
# assigns to each horizon the corresponding training forecast dates for the test date
h_dict = {}
for h in range(1, 5):
    h_dict[h] = [test_date - pd.Timedelta(weeks=4) - pd.Timedelta(weeks=(h - 1)), 
                 test_date - pd.Timedelta(weeks=h)]

In [47]:
h_dict

{1: [Timestamp('2021-03-15 00:00:00'), Timestamp('2021-04-05 00:00:00')],
 2: [Timestamp('2021-03-08 00:00:00'), Timestamp('2021-03-29 00:00:00')],
 3: [Timestamp('2021-03-01 00:00:00'), Timestamp('2021-03-22 00:00:00')],
 4: [Timestamp('2021-02-22 00:00:00'), Timestamp('2021-03-15 00:00:00')]}

In [48]:
def relevant_horizons(d):
    hs = []
    for h in range(1, 5):
        if((d >= h_dict[h][0]) & (d <= h_dict[h][1])):
            hs.append(h)
    return hs

In [49]:
df_files['horizons'] = df_files.timezero.apply(relevant_horizons)

In [50]:
df_files

,filename,model,forecast_date,timezero,horizons
9,data-processed/AIpert-pwllnod/2021-02-22-AIper...,AIpert-pwllnod,2021-02-22,2021-02-22,[4]
10,data-processed/AIpert-pwllnod/2021-03-01-AIper...,AIpert-pwllnod,2021-03-01,2021-03-01,"[3, 4]"
11,data-processed/AIpert-pwllnod/2021-03-08-AIper...,AIpert-pwllnod,2021-03-08,2021-03-08,"[2, 3, 4]"
12,data-processed/AIpert-pwllnod/2021-03-15-AIper...,AIpert-pwllnod,2021-03-15,2021-03-15,"[1, 2, 3, 4]"
13,data-processed/AIpert-pwllnod/2021-03-22-AIper...,AIpert-pwllnod,2021-03-22,2021-03-22,"[1, 2, 3]"
...,...,...,...,...,...
3099,data-processed/epiforecasts-ensemble1/2021-03-...,epiforecasts-ensemble1,2021-03-08,2021-03-08,"[2, 3, 4]"
3100,data-processed/epiforecasts-ensemble1/2021-03-...,epiforecasts-ensemble1,2021-03-15,2021-03-15,"[1, 2, 3, 4]"
3101,data-processed/epiforecasts-ensemble1/2021-03-...,epiforecasts-ensemble1,2021-03-22,2021-03-22,"[1, 2, 3]"
3102,data-processed/epiforecasts-ensemble1/2021-03-...,epiforecasts-ensemble1,2021-03-29,2021-03-29,"[1, 2]"


In [51]:
# only keep relevant training data
df_files = df_files[df_files.horizons.apply(len) > 0]

In [53]:
dfs = []
for _, row in tqdm(df_files.iterrows(), total=df_files.shape[0]):
    VALID_TARGETS = [f"{_} wk ahead inc death" for _ in row['horizons']] + \
                    [f"{_} wk ahead cum death" for _ in row['horizons']] + \
                    [f"{_} wk ahead inc case" for _ in row['horizons']]
    df_temp = pd.read_csv('https://github.com/reichlab/covid19-forecast-hub/raw/master/' + row['filename'],
                         dtype=dtype, parse_dates=parse_dates)
    df_temp = df_temp[df_temp.target.isin(VALID_TARGETS)]
    df_temp['model'] = row['model']
    dfs.append(df_temp)

  0%|          | 0/463 [00:00<?, ?it/s]

In [55]:
df = pd.concat(dfs)

In [57]:
df.shape

(11072397, 8)

# Data Cleaning

In [58]:
df = df[df.location.str.len() == 2]
df = df[~df.location.isin(locations_to_exclude)]

In [59]:
df.location.nunique() # US + 50 states

51

In [60]:
df = df[df.type == 'quantile']

In [61]:
# how many forecasts for each target/model/location? should be 4 for every location
df['no_forecasts'] = df.groupby(['target', 'model', 'location'])['target_end_date'].transform('nunique')

In [64]:
df = df[df.groupby(['target', 'model'])['no_forecasts'].transform('min') == 4].drop(columns='no_forecasts').reset_index(drop=True)

In [186]:
# df = df[df.groupby(['target', 'model', 'location'])['target_end_date'].transform('nunique') == 4]
# would only remove the respective location with <4 forecasts, but we want to remove the model as a whole if in one location <4

In [66]:
df = df[df.groupby(['target', 'model', 'target_end_date'])['location'].transform('nunique') == 51]

In [67]:
df['no_quantiles'] = df.groupby(['model', 'target', 'target_end_date', 'location'])['quantile'].transform('nunique')

df['no_quantiles'] = df.groupby(['target', 'model'])['no_quantiles'].transform('min')

In [72]:
df[(df.target.str.contains('inc case') & (df.no_quantiles == 7))]

,forecast_date,target,target_end_date,quantile,type,value,location,model,no_quantiles


In [73]:
df = df[(df.no_quantiles == 23) | 
        (df.target.str.contains('inc case') & (df.no_quantiles == 7))].drop(columns='no_quantiles').reset_index(drop=True)

In [74]:
df.groupby('target').model.nunique()

target
1 wk ahead cum death    29
1 wk ahead inc death    29
2 wk ahead cum death    28
2 wk ahead inc death    27
3 wk ahead cum death    27
3 wk ahead inc death    27
4 wk ahead cum death    27
4 wk ahead inc death    27
Name: model, dtype: int64

In [75]:
# check if there are models used for training that are not available for the test date
train_models = dict(df.groupby('target').model.unique())

a = [(k, t) for k, v in train_models.items() for t in v]
b = [(k, t) for k, v in available_models.items() for t in v]

[i for i in a if i not in b]

[('1 wk ahead cum death', 'IHME-CurveFit'),
 ('1 wk ahead cum death', 'IUPUI-HkPrMobiDyR'),
 ('1 wk ahead cum death', 'PSI-DRAFT'),
 ('1 wk ahead cum death', 'WalmartLabsML-LogForecasting'),
 ('1 wk ahead inc death', 'IHME-CurveFit'),
 ('1 wk ahead inc death', 'IUPUI-HkPrMobiDyR'),
 ('1 wk ahead inc death', 'MITCovAlliance-SIR'),
 ('1 wk ahead inc death', 'PSI-DRAFT'),
 ('2 wk ahead cum death', 'PSI-DRAFT'),
 ('2 wk ahead cum death', 'UCLA-SuEIR'),
 ('2 wk ahead cum death', 'WalmartLabsML-LogForecasting'),
 ('2 wk ahead inc death', 'PSI-DRAFT'),
 ('2 wk ahead inc death', 'UCLA-SuEIR'),
 ('3 wk ahead cum death', 'PSI-DRAFT'),
 ('3 wk ahead cum death', 'UCLA-SuEIR'),
 ('3 wk ahead inc death', 'PSI-DRAFT'),
 ('3 wk ahead inc death', 'UCLA-SuEIR'),
 ('4 wk ahead cum death', 'PSI-DRAFT'),
 ('4 wk ahead cum death', 'UCLA-SuEIR'),
 ('4 wk ahead inc death', 'PSI-DRAFT'),
 ('4 wk ahead inc death', 'UCLA-SuEIR')]

In [77]:
df = df[df.apply(lambda x: x.model in (available_models[x.target]), axis=1)]

In [78]:
df.groupby('target').model.nunique()

target
1 wk ahead cum death    25
1 wk ahead inc death    25
2 wk ahead cum death    25
2 wk ahead inc death    25
3 wk ahead cum death    25
3 wk ahead inc death    25
4 wk ahead cum death    25
4 wk ahead inc death    25
Name: model, dtype: int64

In [79]:
df.shape

(964206, 8)

In [80]:
models_by_target = pd.DataFrame([(i, k) for i,j in train_models.items() for k in j], 
                  columns=['target','model'])

In [81]:
models_by_target[models_by_target.target == '4 wk ahead cum death']

,target,model
167,4 wk ahead cum death,BPagano-RtDriven
168,4 wk ahead cum death,CEID-Walk
169,4 wk ahead cum death,COVIDhub-baseline
170,4 wk ahead cum death,CU-nochange
171,4 wk ahead cum death,CU-scenario_high
172,4 wk ahead cum death,CU-scenario_low
173,4 wk ahead cum death,CU-scenario_mid
174,4 wk ahead cum death,CU-select
175,4 wk ahead cum death,Covid19Sim-Simulator
176,4 wk ahead cum death,CovidAnalytics-DELPHI


In [88]:
df = df[~df.model.isin(models_to_exclude)]

In [89]:
df

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-02-21,4 wk ahead inc death,2021-03-20,0.010,quantile,3722.68744,US,BPagano-RtDriven
1,2021-02-21,4 wk ahead inc death,2021-03-20,0.025,quantile,4162.47604,US,BPagano-RtDriven
2,2021-02-21,4 wk ahead inc death,2021-03-20,0.050,quantile,4555.62387,US,BPagano-RtDriven
3,2021-02-21,4 wk ahead inc death,2021-03-20,0.100,quantile,5015.98711,US,BPagano-RtDriven
4,2021-02-21,4 wk ahead inc death,2021-03-20,0.150,quantile,5328.61438,US,BPagano-RtDriven
...,...,...,...,...,...,...,...,...
1062733,2021-04-05,1 wk ahead cum death,2021-04-10,0.850,quantile,561091.60000,US,epiforecasts-ensemble1
1062734,2021-04-05,1 wk ahead cum death,2021-04-10,0.900,quantile,561509.00000,US,epiforecasts-ensemble1
1062735,2021-04-05,1 wk ahead cum death,2021-04-10,0.950,quantile,562146.20000,US,epiforecasts-ensemble1
1062736,2021-04-05,1 wk ahead cum death,2021-04-10,0.975,quantile,562489.05000,US,epiforecasts-ensemble1


In [90]:
available_train_models = dict(df.groupby(['target'])['model'].unique())

In [91]:
available_train_models

{'1 wk ahead cum death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'DDS-NBDS',
        'JHUAPL-Bucky', 'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1',
        'MIT_CritData-GBCF', 'MOBS-GLEAM_COVID', 'Microsoft-DeepSTIA',
        'RobertWalraven-ESG', 'SteveMcConnell-CovidComplete',
        'UA-EpiCovDA', 'UCSD_NEU-DeepGLEAM', 'UMass-MechBayes',
        'USC-SI_kJalpha', 'epiforecasts-ensemble1'], dtype=object),
 '1 wk ahead inc death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'DDS-NBDS',
        'JHUAPL-Bucky', 'Karlen-pypm', 'LANL-GrowthRate', 'LNQ-ens1',
        'MIT_CritData-GBCF', 'MOBS-GLEAM_COVID', 'Microsoft-DeepSTIA',
        'RobertWalraven-ESG', 'SteveMcConnell-CovidComplete',
        'UA-EpiCovDA', 'UCSD_NEU-DeepGLEAM', 'UMass-MechBayes',
        'USC-SI_kJalpha', 'epiforecasts-ensemble1'], dtype=obj

In [109]:
df_test.shape

(206448, 8)

In [98]:
df_test = df_test[df_test.apply(lambda x: x.model in (available_train_models[x.target]), axis=1)]

In [110]:
df_test.head()

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-04-11,1 wk ahead inc death,2021-04-17,0.010,quantile,3287.41123,US,BPagano-RtDriven
1,2021-04-11,1 wk ahead inc death,2021-04-17,0.025,quantile,3571.70953,US,BPagano-RtDriven
2,2021-04-11,1 wk ahead inc death,2021-04-17,0.050,quantile,3825.85723,US,BPagano-RtDriven
3,2021-04-11,1 wk ahead inc death,2021-04-17,0.100,quantile,4123.45586,US,BPagano-RtDriven
4,2021-04-11,1 wk ahead inc death,2021-04-17,0.150,quantile,4325.55159,US,BPagano-RtDriven


In [114]:
df.groupby(['target'])['model'].unique()

target
1 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
1 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
Name: model, dtype: object

# Export

In [115]:
df.to_csv('data/ensemble_data/df_train_2021-04-12.csv', index=False)

In [116]:
df_test.to_csv('data/ensemble_data/df_test_2021-04-12.csv', index=False)